In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummaryX import summary
from UnaryComputingSim.sw.kernel.linear import UnaryLinear


In [2]:
import os
cwd = "/Users/wudidaizi/Project/stochasticSim/pytorchSim"
print(cwd)
model_path = cwd+"/saved_model_state_dict_8"
print(model_path)

/Users/wudidaizi/Project/stochasticSim/pytorchSim
/Users/wudidaizi/Project/stochasticSim/pytorchSim/saved_model_state_dict_8


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
# MNIST data loader
transform=transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=4)

testset = torchvision.datasets.MNIST(root=cwd+'/data/mnist', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=1, num_workers=4)

0it [00:00, ?it/s]

9920512it [00:02, 4862628.48it/s]                             


Extracting /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw


  0%|          | 0/28881 [00:00<?, ?it/s]

32768it [00:00, 119945.71it/s]           
  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw


1654784it [00:01, 1545323.76it/s]                           
0it [00:00, ?it/s]

Extracting /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw


8192it [00:00, 46508.19it/s]            

Extracting /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /Users/wudidaizi/Project/stochasticSim/pytorchSim/data/mnist/MNIST/raw
Processing...
Done!


In [5]:
 class LeNet(nn.Module):

    def __init__(self):
        super(LeNet, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = self.conv1(x)
        x = torch.clamp(x, -1, 1)
        x = F.avg_pool2d(x, (2, 2))
        x = F.relu(x)
        # If the size is a square you can only specify a single number
        x = self.conv2(x)
        x = torch.clamp(x, -1, 1)
        x = F.avg_pool2d(x, (2, 2))
        x = F.relu(x)
        
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = torch.clamp(x, -1, 1)
        x = F.relu(self.fc2(x))
        x = torch.clamp(x, -1, 1)
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(32*32, 512)
        self.fc1_drop = nn.Dropout(0.6)
        self.fc2 = nn.Linear(512, 256)
        self.fc2_drop = nn.Dropout(0.6)
        self.fc3 = nn.Linear(256, 10)

    def forward(self, x):
        x = x.view(-1, 32*32)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)        
        x = torch.clamp(x, -1, 1)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        x = torch.clamp(x, -1, 1)
        return F.log_softmax(self.fc3(x), dim=1)
    
model = Net()
model.to(device)


Net(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc1_drop): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc2_drop): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [6]:
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

Net(
  (fc1): Linear(in_features=1024, out_features=512, bias=True)
  (fc1_drop): Dropout(p=0.6, inplace=False)
  (fc2): Linear(in_features=512, out_features=256, bias=True)
  (fc2_drop): Dropout(p=0.6, inplace=False)
  (fc3): Linear(in_features=256, out_features=10, bias=True)
)

In [7]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(images.size())
print('Accuracy of the network on the 10000 test images: %f %%' % (
    100 * correct / total))

torch.Size([1, 1, 32, 32])
Accuracy of the network on the 10000 test images: 94.550000 %


In [ ]:
fc400 = nn.Linear(400, 120)
fc400.weight = lenet.fc1.weight
fc400.bias = lenet.fc1.bias
ufc400 = UnaryLinear(400, 120, 256, lenet.fc1.weight, lenet.fc1.bias)
print(fc400.weight.size())

inVec = (((torch.rand(400, 120) * 2 - 1))*256).floor()/256
# print(inVec)
outVec = fc400(inVec)
outVec.clamp_(-1.,1.)

bsGen = BSGen(inVec, mode="Sobol", bipolar=True)
# bsGen = BitStreamGen(inVec,mode="Race")
pp = ProgressivePrecision(outVec, auto_print=False,bipolar=True)
ipp = ProgressivePrecision(inVec, auto_print=False,bipolar=True)

with torch.no_grad():
    for i in range(256):
        input = bsGen.Gen()
#         print(ipp.Monitor(input))
        output = ufc400(input)
        pp.Monitor(output)


